In [1]:
#essential imports
import pandas as pd
import numpy as np

import datetime as dt
import time

# Strict JSON conversion
import json 
import gzip 
 
startyear=1990
startmonth=1

cps = 4
# how many month before the death of the artist are used as feature 
nbMonthBeforeDeath = 4

datapath = 'DATA/review_filtered/'
f_sbj_reviews = datapath+'music_reviews_matched_'+str(cps)+'cps.csv'
f_ctl_reviews = datapath+'music_reviews_matched_'+str(cps)+'cps_ctrls.csv'

# Load data

reviews related to subject (=dead) artists

In [2]:
sbj_reviews = pd.read_csv(f_sbj_reviews).drop('Unnamed: 0',axis=1)
sbj_reviews = sbj_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'})
sbj_reviews.head(3)

,asin,reviewText,summary,strReviewTime,artist name,death date,birth date
0,B000KB9Y6U,I bought this item because it did not say vyna...,Should be more descriptive in the title.,2013-05-23,paul delph,1996-05-21,1957
1,B000KB9Y6U,Chrissy Faith is wonderful I must say!! What ...,You should listen this!!,2013-11-09,paul delph,1996-05-21,1957
2,B000KB9Y6U,this product came quickly and there were no is...,great music,2013-04-26,paul delph,1996-05-21,1957


reviews related to control (=dead) artists

In [3]:
ctl_reviews = pd.read_csv(f_ctl_reviews).drop('Unnamed: 0',axis=1)
ctl_reviews = ctl_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'})
ctl_reviews.head(3)

,asin,reviewText,summary,strReviewTime,artist name,death date,birth date
0,B0002VYEHW,"In a reggae universe gone mad for dancehall, T...",Plain good reggae,2005-08-23,tike jah fakoly,unknown,unknown
1,B0002VYEHW,"Tiken Jah's ""Coup de Gueule"" is on the continu...","MBafoh Pian!!!( I Shall Say It Loud and Sound,...",2006-03-10,tike jah fakoly,unknown,unknown
2,B0040AT66U,This is very touching little opera. The libret...,A Mini Masterpiece,2013-05-28,patricia risley,unknown,unknown


# Pre-process data 

- transform string dates into formats that are easier for us to work with
- keep on the side the 'death date' <-> 'artist name relation 

Convert dates to year and extract and set aside actor<-> deathdates

In [4]:
# returns as an int the month of the date passed as argument. Apply to a serie !
def strdateToMonth(date) : 
    year = int(str(date)[:4])
    month = int(str(date)[5:7])
    assert(month <= 12 and month >= 1)
    return (month - startmonth) + (year - startyear)*12 

# returns as an int the year of the date passed as argument. Apply to a serie !
def strdateToYear(date) : 
    return int(str(date)[:4])

# SUBJECTS
sbj_reviews['reviewMonth'] = sbj_reviews['strReviewTime'].apply(strdateToMonth)
sbj_reviews['reviewYear'] = sbj_reviews['strReviewTime'].apply(strdateToYear)
sbj_reviews['reviewTime'] = pd.to_datetime(sbj_reviews['strReviewTime'])
sbj_reviews['death month'] = sbj_reviews['death date'].apply(strdateToMonth)
sbj_reviews['death year'] = sbj_reviews['death date'].apply(strdateToYear)
sbj_reviews['death date'] = pd.to_datetime(sbj_reviews['death date'])

# CONTROLS
ctl_reviews['reviewMonth'] = ctl_reviews['strReviewTime'].apply(strdateToMonth)
ctl_reviews['reviewYear'] = ctl_reviews['strReviewTime'].apply(strdateToYear)
ctl_reviews['reviewTime'] = pd.to_datetime(ctl_reviews['strReviewTime'])

Set aside the 'death date' <-> 'artist name relation 

In [5]:
deathdates = sbj_reviews.groupby(['artist name','death date', 'death year', 'death month']).count()\
                            .reset_index()[['artist name', 'death date', 'death year', 'death month']]
deathdates.head()

,artist name,death date,death year,death month
0,adrian borland,1999-04-26,1999,111
1,akira ifukube,2006-02-08,2006,193
2,al hirt,1999-04-27,1999,111
3,al martino,2009-10-13,2009,237
4,alain bashung,2009-03-14,2009,230


Extract controls names

In [6]:
#TODO : remove
#ctrl_names = list(set(controls_actors['actor name']))\
#sbj_names = list(deathdates['actor name'])

# filtering

### 'mentionned' Boolean

We want to distinguish which artists are frequently mentionned in their reviews; i.e, those that are popular, as opposed to those that just appear to play/sing with some other, more popular artist.
We add the boolean ['mentionned'] that tells if the artist listed with the review is mentionnned in the text of the review.

In [7]:
thresh_pop = 1

def mentionsName(row) : 
    if type(row['reviewText']) is str :
        names = row['artist name'].split(' ')
        text =  row['reviewText'].lower()
        # can mention only name or surname
        #return row['actor name'] in row['reviewText']
        for name in names : 
            if name in text : 
                return True
        return False
    else : return False

In [8]:
## Treat subjects
##
sbj_reviews['mentionned'] = sbj_reviews.apply(mentionsName, axis=1)
sbj_reviews.shape, sbj_reviews[sbj_reviews['mentionned']==True].shape

((320780, 13), (96181, 13))

In [9]:
## Treat controls
##
ctl_reviews['mentionned'] = ctl_reviews.apply(mentionsName, axis=1)
ctl_reviews.shape, ctl_reviews[ctl_reviews['mentionned']==True].shape

((119120, 11), (38852, 11))

Not necessarily important for later : the list of the most-mentionned subjects and controls

In [10]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
sbj_popular = sbj_reviews[sbj_reviews['mentionned']==True]
sbj_popular = sbj_popular.groupby('artist name').count()['asin']
print(sbj_reviews.shape[0], " reduced to", sbj_popular.shape)
sbj_popular.sort_values(ascending=False)[:5]

320780  reduced to (348,)


artist name
the rev            9136
johnny cash        4492
big l              2960
whitney houston    2392
john denver        2139
Name: asin, dtype: int64

In [11]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
ctl_popular = ctl_reviews[ctl_reviews['mentionned']==True]
ctl_popular = ctl_popular.groupby('artist name').count()['asin']
print(ctl_reviews.shape[0], " reduced to", ctl_popular.shape)
ctl_popular.sort_values(ascending=False)[:5]

119120  reduced to (1258,)


artist name
elvis presley      6307
andrea bocelli     2499
george harrison    2337
diana ross         1614
toni braxton       1125
Name: asin, dtype: int64

### Date filtering

We're only interested in the reviews of subjects that died in a recent time period because amazon's popularity is recent, so we'll have much more amazon data after year 2000. Therefore, we will only keep the reviews for which the subject died within a 2003-2014 time period, otherwise, the artist died too long ago.

In [19]:
start_year = 2004
end_year = 2014

print('original size : ', sbj_reviews.shape)
sbj_reviews = sbj_reviews[(sbj_reviews['death year']>=start_year) \
                        & (sbj_reviews['death year']<=end_year)]
print('filtered size : ', sbj_reviews.shape)
sbj_reviews.head(3)

original size :  (320780, 13)
filtered size :  (166352, 13)


,asin,reviewText,summary,strReviewTime,artist name,death date,birth date,reviewMonth,reviewYear,reviewTime,death month,death year,mentionned
154428,B00004XQMG,"Great Album. Mix of Upbeat, w/reserved. Defini...",The French Tom Jones/avec plus de jazz,2004-06-14,claude nougaro,2004-03-04,1929,173,2004,2004-06-14,170,2004,False
154429,B00004YWCV,The legendary Charles Aznavour was in his mid ...,&quot;Entertainer of the Century&quot;,2001-11-21,claude nougaro,2004-03-04,1929,142,2001,2001-11-21,170,2004,False
154430,B00004YWCV,UNBELIEVABLE CD INCREDIBLE CONDITION AS IF IT ...,FANTASTIC,2013-06-26,claude nougaro,2004-03-04,1929,281,2013,2013-06-26,170,2004,False


# Extract features used to match controls with subjects

We will use as features the number of review per month for a 4 months before the death of the subject. We obtain a features vector that we will try to match best between control and subjects.

Since the controls didn't die however, and since the amount of amazon reviews is generated increasingly fast as time passses, we will this feature for every control, for every subject we try to match it to, using the subject's death date as reference point and using the control's number of reviews one month before, two months before, and so on.

In [20]:
# relative count of months before death (-1, -2, ...)
relMonthBeforeDeath = [-i for i in range(1, nbMonthBeforeDeath+1)]
# string matching the feature's column in the subject's dataframe
relFeaturesStr = { i : 'revs'+str(i) for i in relMonthBeforeDeath}

In [21]:
# SUBJECTS COUNTS BY MONTH
sbj_reviews_monthly = sbj_reviews.groupby(['artist name','reviewMonth']).count()


''' returns the number of reviews for the subject in row, 'relMonthNb' months before his death month
'''
def getreview_month_subjects(row, relMonthNb) : 
    actor = row['artist name']
    deathMonth = int(row['death month'])
    try : 
        val = sbj_reviews_monthly.loc[actor, int(deathMonth+relMonthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0
    

    
sbj_monthly_feats = pd.DataFrame(deathdates[['artist name', 'death month']])
for i in relMonthBeforeDeath :
    sbj_monthly_feats[relFeaturesStr[i]] = sbj_monthly_feats\
                                    .apply(lambda row : getreview_month_subjects(row, i), axis=1)

## Compute the norm of the feature
##
def feature_norm(row) : 
    feature = [row[relFeaturesStr[i]] for i in relMonthBeforeDeath]
    return np.linalg.norm(feature)

sbj_monthly_feats['featureNorm'] = sbj_monthly_feats.apply(feature_norm, axis=1)
sbj_monthly_feats['death date'] = deathdates['death date']
        
sbj_monthly_feats.sort_values(by='featureNorm', ascending=False).head()

,artist name,death month,revs-1,revs-2,revs-3,revs-4,featureNorm,death date
290,proof,195,107,114,112,155,247.010121,2006-04-11
121,george jones,279,135,102,100,86,214.534380,2013-04-26
171,jj cale,282,25,50,99,136,177.262517,2013-07-26
335,the rev,239,52,52,84,65,129.185913,2009-12-28
222,lou reed,285,79,39,37,29,99.859902,2013-10-27


# Matching

Prepare the control's dataframe for matching, and create a function that computes the features for subjects

In [22]:
ctl_reviews_sorted = ctl_reviews.groupby(['artist name','reviewMonth'])\
                                .count()['asin']\
                                .to_frame().reset_index()
        
ctl_reviews_sorted.sort_values(by=['artist name', 'asin']).head()        

,artist name,reviewMonth,asin
0,a campra,185,1
1,a khachaturian,195,1
2,a khachaturian,197,1
3,a khachaturian,208,1
4,a khachaturian,222,1


In [23]:
''' returns the number of reviews for the control in row, 
    looking in 'musiciansDataframe', 'relMonthNb' months before 'deathMonth'
'''
def getreview_month_controls(row, monthNb, musiciansDataframe, deathMonth) : 
    actor = row['artist name']
    try : 
        val = musiciansDataframe.loc[actor, int(deathMonth+monthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0

### Matching algorithm : 
   
   
Match is a greedy function, that given a threshold and a subject, will iterate over the controls, compare the features, and if the difference is below a given threshold, call it a match. If the threshold cannot be met, it returns that it couldn't match.
We call this function over all the subjects, increasing the threshold until a match can be found. 

In [24]:
# helper to obtain a value after indexing on a dataframe
def unwrap(subjectStuff) : 
    return subjectStuff.tolist()[0]

def match(artistname, threshold, ctrl_revs) : 

    # extract the useful features for the subject 
    subject = sbj_monthly_feats[sbj_monthly_feats['artist name'] == artistname]
    deathmonth = int(subject['death month'])
    sbjname = unwrap(subject['artist name'])
    feature = [unwrap(subject[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    #print(sbj_actor)
    #print('feature : ', feature)
    
    for control in ctrl_revs.groupby('artist name').count().reset_index().iterrows() : 
        
        # iterrows returns tuple (0, row)
        idx = control[0]
        control = control[1]
        
        # Compute feature
        ctrl_feature = [getreview_month_controls(control, i, 
                                        ctrl_revs.set_index(['artist name', 'reviewMonth']), 
                                        deathMonth=deathmonth) \
                        for i in relMonthBeforeDeath]
        
        #print('   ctrl_feature : ', ctrl_feature )
        
        diff = np.linalg.norm(np.array(feature) - np.array(ctrl_feature))
        
        #print('    diff : ', diff)
        
        if diff <= threshold : 
            matched_name = control['artist name']
            matched_count = ctrl_feature
            
            #ctrl_revs = ctrl_revs[ctrl_revs['actor name'].isin([matched_name])==False]
            ctrl_revs = ctrl_revs[ctrl_revs['artist name']!=matched_name]
            
            print('    ', sbjname, ' matched with ', matched_name)
            print('    ', feature, ' ~ ' , ctrl_feature)
            return matched_name, ctrl_feature, ctrl_revs
    
    print('    1 iteration completed over all controls')
    return 0, 0, ctrl_revs  


In [25]:
ctrl_revs = ctl_reviews_sorted.copy()
sbj_monthly_feats_sorted = sbj_monthly_feats.sort_values(by='featureNorm', ascending=False)

sbj_list = []
sbj_feat_list = []
sbj_death_list = []
ctl_list = []
ctl_feat_list = []

for idx, subject_name in enumerate(sbj_monthly_feats_sorted['artist name']):
    
    if idx%50 == 0: 
        print(idx, " subjects out of ", len(sbj_monthly_feats['artist name']))
    
    # extract the feature for the subject and use it to set the threshold
    subject = sbj_monthly_feats_sorted[sbj_monthly_feats_sorted['artist name'] == subject_name]
    feature = [unwrap(subject[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    sbjname = unwrap(subject['artist name'])
    print(sbjname, '-', feature)
    
    featureNorm = subject['featureNorm']
    thr = int(8 + 0.12*featureNorm)
    mtch = 0

    while mtch == 0:
        
        name, ctrl_feature, ctrl_revs_ = match(sbjname,thr,ctrl_revs)
        
        # Scramble the order of iteration of controls for the next iteration
        ctrl_revs = ctrl_revs_.sample(frac=1) 
        
        if name != 0:
            
            #print(subject_name+' matched with '+name)
            sbjDeathDate = unwrap(subject['death date'])
            
            sbj_list.append(sbjname)
            sbj_feat_list.append(feature)
            sbj_death_list.append(sbjDeathDate)
            ctl_list.append(name)
            ctl_feat_list.append(ctrl_feature)
            
            mtch += 1
            
        
        thr += int(0.3*thr + 1)
        print('  thr increased: ', thr)
            
print('DONE')

0  subjects out of  368
proof - [107, 114, 112, 155]
    1 iteration completed over all controls
  thr increased:  49
    1 iteration completed over all controls
  thr increased:  64
    1 iteration completed over all controls
  thr increased:  84
    1 iteration completed over all controls
  thr increased:  110
     proof  matched with  elvis presley
     [107, 114, 112, 155]  ~  [106, 72, 57, 70]
  thr increased:  144
george jones - [135, 102, 100, 86]
    1 iteration completed over all controls
  thr increased:  43
    1 iteration completed over all controls
  thr increased:  56
    1 iteration completed over all controls
  thr increased:  73
     george jones  matched with  george harrison
     [135, 102, 100, 86]  ~  [71, 81, 92, 79]
  thr increased:  95
jj cale - [25, 50, 99, 136]
    1 iteration completed over all controls
  thr increased:  38
     jj cale  matched with  steve winwood
     [25, 50, 99, 136]  ~  [37, 80, 102, 144]
  thr increased:  50
the rev - [52, 52, 84, 65]
 

     lorin maazel  matched with  alfred hitchcock
     [8, 8, 9, 8]  ~  [2, 8, 6, 3]
  thr increased:  12
isaac hayes - [7, 10, 7, 8]
     isaac hayes  matched with  bebel gilberto
     [7, 10, 7, 8]  ~  [8, 8, 3, 5]
  thr increased:  12
lou rawls - [12, 9, 4, 3]
     lou rawls  matched with  keith murray
     [12, 9, 4, 3]  ~  [6, 6, 4, 1]
  thr increased:  12
patti page - [10, 11, 4, 2]
     patti page  matched with  eliane elias
     [10, 11, 4, 2]  ~  [7, 7, 1, 3]
  thr increased:  12
billy preston - [3, 6, 8, 10]
     billy preston  matched with  dax riggs
     [3, 6, 8, 10]  ~  [1, 0, 3, 7]
  thr increased:  12
michael brecker - [6, 10, 3, 5]
     michael brecker  matched with  dietrich fische
     [6, 10, 3, 5]  ~  [1, 9, 0, 0]
  thr increased:  12
joe zawinul - [4, 8, 0, 8]
     joe zawinul  matched with  christian fennesz
     [4, 8, 0, 8]  ~  [0, 1, 0, 4]
  thr increased:  12
les paul - [4, 2, 4, 10]
     les paul  matched with  dj khaled
     [4, 2, 4, 10]  ~  [4, 2, 2, 2]
 

     anita oday  matched with  alfred lessing
     [1, 0, 2, 2]  ~  [0, 0, 0, 0]
  thr increased:  11
100  subjects out of  368
clarence clemons - [2, 0, 2, 0]
     clarence clemons  matched with  alice gomez
     [2, 0, 2, 0]  ~  [1, 0, 0, 1]
  thr increased:  11
carlo bergonzi - [1, 1, 2, 1]
     carlo bergonzi  matched with  alicia dara
     [1, 1, 2, 1]  ~  [0, 0, 0, 0]
  thr increased:  11
odetta - [1, 0, 1, 2]
     odetta  matched with  alicia zizzo
     [1, 0, 1, 2]  ~  [0, 0, 0, 0]
  thr increased:  11
joan sutherland - [1, 1, 2, 0]
     joan sutherland  matched with  alisa fineman
     [1, 1, 2, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
renata tebaldi - [0, 1, 1, 2]
     renata tebaldi  matched with  alison crockett
     [0, 1, 1, 2]  ~  [0, 0, 1, 0]
  thr increased:  11
jerry reed - [0, 1, 2, 1]
     jerry reed  matched with  alison crum
     [0, 1, 2, 1]  ~  [0, 0, 0, 0]
  thr increased:  11
henri salvador - [2, 1, 0, 0]
     henri salvador  matched with  alix dobkin
     [2,

     nicolette larson  matched with  athena reich
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
nat adderley - [0, 0, 0, 0]
     nat adderley  matched with  atlantean kodex
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
michael hutchence - [0, 0, 0, 0]
     michael hutchence  matched with  atlantis ita
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
michael hedges - [0, 0, 0, 0]
     michael hedges  matched with  au ras au ras
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
marcel dadi - [0, 0, 0, 0]
     marcel dadi  matched with  augustin dumay
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
nazia hassan - [0, 0, 0, 0]
     nazia hassan  matched with  aurore bucher
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
norman smith - [0, 0, 0, 0]
     norman smith  matched with  ayaka ide
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
natasja saad - [0, 0, 0, 0]
     natasja saad  matched with  ayako oshima
     [0, 0, 0, 0]  ~  [0

     robert simpson  matched with  brian zeger
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
roger troutman - [0, 0, 0, 0]
     roger troutman  matched with  brianna haynes
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
rosalyn tureck - [0, 0, 0, 0]
     rosalyn tureck  matched with  bridget stjohn
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
rose maddox - [0, 0, 0, 0]
     rose maddox  matched with  britten sinfonia
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
rosemary clooney - [0, 0, 0, 0]
     rosemary clooney  matched with  brooklyn rundfunk orkestrata
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
roy rogers - [0, 0, 0, 0]
     roy rogers  matched with  bruce eskovitz
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
rozz williams - [0, 0, 0, 0]
     rozz williams  matched with  bruce roberts
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
manolo escobar - [0, 0, 0, 0]
     manolo escobar  matched with  bruno leonard

     boxcar willie  matched with  chris irwin
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
bradley nowell - [0, 0, 0, 0]
     bradley nowell  matched with  christian andreason
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
brian connolly - [0, 0, 0, 0]
     brian connolly  matched with  christian blackshaw
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
bulat okudzhava - [0, 0, 0, 0]
     bulat okudzhava  matched with  christian du plessis
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
burton lane - [0, 0, 0, 0]
     burton lane  matched with  christian wolff
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
carl perkins - [0, 0, 0, 0]
     carl perkins  matched with  christiane eda-pierre
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
carl wilson - [0, 0, 0, 0]
     carl wilson  matched with  christof fischeser
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
300  subjects out of  368
frank de vol - [0, 0, 0, 0]
     frank de 

     iannis xenakis  matched with  darek oleszkiewicz
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
irving caesar - [0, 0, 0, 0]
     irving caesar  matched with  dariusz skoraczewski
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
isaac stern - [0, 0, 0, 0]
     isaac stern  matched with  darlene ahuna
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
israel kamakawiwo?ole - [0, 0, 0, 0]
     israel kamakawiwo?ole  matched with  darlene mccoy
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
izumi sakai - [0, 0, 0, 0]
     izumi sakai  matched with  darrell mcfadden
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
jacob druckman - [0, 0, 0, 0]
     jacob druckman  matched with  darren ockert
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
jam master jay - [0, 0, 0, 0]
     jam master jay  matched with  dave armstrong
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
  thr increased:  11
jeff buckley - [0, 0, 0, 0]
     jeff buckley  matched with  dave ath

# Results

In [26]:
res = pd.DataFrame()

res['sbj'] = sbj_list
res['sbj_feats'] = sbj_feat_list
res['death date'] = sbj_death_list
res['ctl_feats'] = ctl_feat_list
res['ctl'] = ctl_list

res.head()

,sbj,sbj_feats,death date,ctl_feats,ctl
0,proof,"[107, 114, 112, 155]",2006-04-11,"[106, 72, 57, 70]",elvis presley
1,george jones,"[135, 102, 100, 86]",2013-04-26,"[71, 81, 92, 79]",george harrison
2,jj cale,"[25, 50, 99, 136]",2013-07-26,"[37, 80, 102, 144]",steve winwood
3,the rev,"[52, 52, 84, 65]",2009-12-28,"[16, 23, 25, 26]",daniel lanois
4,lou reed,"[79, 39, 37, 29]",2013-10-27,"[42, 43, 22, 32]",kacey musgraves


In [27]:
# TODO - REMOVE
'''
ctrl_features = pd.DataFrame(ctrl_list, columns=['ctrl_name'])
ctrl_features['ctrl_reviews'] = count_list

res = pd.concat([ctrl_features.reset_index(), sorted_subjects.reset_index()], axis=1)\
            .drop('index', axis=1)
res['error'] = np.abs(res['ctrl_reviews'] - res['reviews'])/(2*(1+res['ctrl_reviews'] + res['reviews']))

res.sort_values(by='error', ascending=False).head()
'''

"\nctrl_features = pd.DataFrame(ctrl_list, columns=['ctrl_name'])\nctrl_features['ctrl_reviews'] = count_list\n\nres = pd.concat([ctrl_features.reset_index(), sorted_subjects.reset_index()], axis=1)            .drop('index', axis=1)\nres['error'] = np.abs(res['ctrl_reviews'] - res['reviews'])/(2*(1+res['ctrl_reviews'] + res['reviews']))\n\nres.sort_values(by='error', ascending=False).head()\n"

In [28]:
# TODO - REMOVE
#ctrl_features.hist(column='ctrl_reviews', figsize=[10,10], bins=100)

In [29]:
# TODO - REMOVE
#subjects_feats.hist(column='reviews', figsize=[10,10], bins=100)

# Export results

In [30]:
subjects_nb = str(res.shape[0]-1)
sbj_reviews.to_csv('reviews_subjects_'+subjects_nb+'_vectormatched.csv', index=False)
ctl_reviews.to_csv('reviews_controls_'+subjects_nb+'_vectormatched.csv', index=False)
res.to_csv('matchings_'+subjects_nb+'_vectormatched.csv', index=False)